In [ ]:
from __future__ import annotations

%pip install uncertainties

from io import StringIO

import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import scipy.optimize
from IPython.display import Markdown
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from uncertainties import ufloat

In [ ]:
def dho_func(
    time: npt.NDArray,
    amplitude_init: float,
    angular_frequency: float,
    damping_term: float,
    phase: float,
    offset: float,
) -> npt.NDArray:
    return (
        dho_amp(time, amplitude_init, damping_term)
        * dho_wave(time, angular_frequency, phase)
        + offset
    )


def dho_amp(
    time: npt.NDArray,
    amplitude_init: float,
    damping_term: float,
) -> npt.NDArray:
    return amplitude_init * np.exp(-damping_term * time / 2)


def dho_wave(
    time: npt.NDArray,
    angular_frequency: float,
    phase: float,
) -> npt.NDArray:
    return np.cos(angular_frequency * time + phase)

Copy your data from the text file and paste it into the `raw_data` variable bellow.

In [ ]:
raw_data = """Vernier Format 2
lab2b.txt 8/19/2023 2:29:48
Run 1
Time	Position	Velocity	Acceleration
T	P	V	A
s	m	m/s	m/s^2

0.00	0.284	-0.227	0.413
...
30.00	0.275	0.004	-0.087
Vernier Format 2
lab2b.txt 8/19/2023 2:29:48
Run 2
Time	Position	Velocity	Acceleration
T	P	V	A
s	m	m/s	m/s^2


"""

df = pd.read_csv(
    StringIO(raw_data),
    sep="\t",
    header=5,
    names=("t", "x", "v", "a"),
    skipfooter=8,
    index_col=False,
    engine="python",
)
display(df)

fig, ax = plt.subplots()
fig.tight_layout()
ax.set_title(R"Position vs. Time Measurements")
ax.set_xlabel(R"Time, $t$ $\left[\text{s}\right]$")
ax.set_ylabel(R"Position, $x$ $\left[\text{m}\right]$")
ax.plot(df.t, df.x, ".-");

In [ ]:
p0: tuple[float, float, float, float, float] = (
    0.01, # amplitude_init
    10.0, # angular_frequency
    0.01, # damping_term
    0.0, # phase
    0.0, # offset
)
popt: npt.NDArray
pcov: npt.NDArray
infodict: dict
mesg: str
ier: int
popt, pcov, infodict, mesg, ier = scipy.optimize.curve_fit(
    dho_func,
    df.t,
    df.x,
    p0,
    full_output=True,
)

display(Markdown("**Curve Fit Message:**"))
print(mesg)

perr = np.sqrt(np.diag(pcov))

display(Markdown(f"**Condition Number of Covariance Matrix:** {np.linalg.cond(pcov)}"))

amplitude_init = ufloat(popt[0], perr[0])
angular_frequency = ufloat(popt[1], perr[1])
damping_term = ufloat(popt[2], perr[2])
phase = ufloat(popt[3], perr[3])
offset = ufloat(popt[4], perr[4])
display(Markdown(fR"""
Fit function:

$$
    x\left(t\right)
    = A_0 e^{{-\gamma t / 2}} \cos\left(\omega t + \varphi\right) + x_0
$$

Amplitude (initial): $$A_0 = {amplitude_init:L}\,\text{{m}}$$
Angular frequency: $$\omega = {angular_frequency:L}\,\text{{rad}}/\text{{s}}$$
Damping term: $$\gamma = {damping_term:L}\,\text{{rad}}/\text{{s}}$$
Phase: $$\varphi = {phase:L}\,\text{{rad}}$$
Offset: $$x_0 = {offset:L}\,\text{{m}}$$"""))

t_curve = np.linspace(np.min(df.t), np.max(df.t), (df.t.count() - 1) * 10 + 1)
df["res"] = df.x - dho_func(df.t, *popt)

fig: Figure
axs: tuple[Axes, Axes]
fig, axs = plt.subplots(2, 1, sharex=True)
fig.tight_layout()
axs[-1].set_xlabel(R"Time, $t$ $\left[\text{s}\right]$")
axs[0].set_ylabel(R"Displacement, $x$ $\left[\text{m}\right]$")
axs[0].scatter(df.t, df.x, s=1, color="C0", label=R"measured", zorder=2.1)
axs[0].plot(t_curve, dho_func(t_curve, *popt), color="k", label=R"fit", linewidth=0.5)
axs[0].legend()
axs[1].set_ylabel(R"Residual $\left[\text{m}\right]$")
axs[1].scatter(df.t, df.res, s=1, color="r", label=R"residual")
axs[1].legend();

See [scipy.optimize.curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) and [numpy.linalg.cond](https://numpy.org/doc/2.1/reference/generated/numpy.linalg.cond.html) for details.